In [32]:
#Siamese Architecture_CNN
#keras_1.1.2_theano_backend
#Dropout=0.5
#epoch=10
#train:validation=8:2
#Using Glove for words

#parameters for the Model
#MAX_SEQUENCE_LENGTH = 200
#MAX_NB_WORDS = 40000
#EMBEDDING_DIM = 100
#VALIDATION_SPLIT = 0.2

#Stopped at 6
#didnot improve after 3

In [2]:
#"image_dim_ordering": "th"\r\n

In [3]:
import os
with open(os.path.expanduser('~')+'/.keras/keras.json','w') as f:
    new_settings = """{\r\n
    "image_dim_ordering": "th",\r\n
    "image_data_format": "channels_first",\n
    "epsilon": 1e-07,\r\n
    "floatx": "float32",\r\n
    "backend": "theano"\r\n
    }"""
    f.write(new_settings)

import keras

Using Theano backend.


In [4]:
from keras import backend as K

In [5]:
K.image_dim_ordering()

'th'

In [6]:
K.backend()

'theano'

In [7]:
import pandas as pd
import nltk
import numpy as np
import gensim

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializations
from keras.callbacks import ModelCheckpoint

from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
import numpy.random as rng
import numpy as np
import os
import dill as pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [9]:
#from keras import backend as K
#K.set_image_dim_ordering('th')

In [10]:
#parameters for the Model
MAX_SEQUENCE_LENGTH = 200
MAX_NB_WORDS = 40000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [11]:
train_DF=pd.read_csv('train.csv',index_col='id')
train_DF.reset_index(drop=True,inplace=True)
train_DF['question1'] = train_DF['question1'].astype(str)
train_DF['question2'] = train_DF['question2'].astype(str)

In [12]:
import re
numbers=re.compile(r'(\d+\.\d+|\d+)')
def cleanText(text):
    return numbers.sub('N',text)

In [13]:
texts = []
questions1=[]
questions2=[]
duplicates=[]

for idx in range(train_DF.question1.shape[0]):
    text=train_DF.question1[idx]
    text=cleanText(text)
    texts.append(text)
    questions1.append(text)
    
    text=train_DF.question2[idx]
    text=cleanText(text)
    texts.append(text)
    questions2.append(text)
    
    duplicates.append(train_DF.is_duplicate[idx])

In [14]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

Total 90278 unique tokens.


In [15]:
questions1_sequences = tokenizer.texts_to_sequences(questions1)
questions2_sequences = tokenizer.texts_to_sequences(questions2)

In [16]:
questions1_sequences = pad_sequences(questions1_sequences, maxlen=MAX_SEQUENCE_LENGTH)
questions2_sequences = pad_sequences(questions2_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [17]:
questions2_sequences.shape

(404290, 200)

In [18]:
duplicates_1=duplicates
duplicates = to_categorical(np.asarray(duplicates))
print('Shape of label tensor:', duplicates.shape)

Shape of label tensor: (404290, 2)


In [19]:
indices = np.arange(questions1_sequences.shape[0])
np.random.shuffle(indices)

questions1_sequences = questions1_sequences[indices]
questions2_sequences = questions2_sequences[indices]
duplicates = duplicates[indices]

nb_validation_samples = int(VALIDATION_SPLIT * questions1_sequences.shape[0])

questions1_sequences_train = questions1_sequences[:-nb_validation_samples]
questions2_sequences_train = questions2_sequences[:-nb_validation_samples]
duplicates_train = duplicates[:-nb_validation_samples]

questions1_sequences_val = questions1_sequences[-nb_validation_samples:]
questions2_sequences_val = questions2_sequences[-nb_validation_samples:]
duplicates_val = duplicates[-nb_validation_samples:]

In [20]:
print('Traing and validation set number of positive and negative reviews')
print(duplicates_train.sum(axis=0))
print(duplicates_val.sum(axis=0))

Traing and validation set number of positive and negative reviews
[ 203981.  119451.]
[ 51046.  29812.]


In [21]:
embeddings_index = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors.' % len(embeddings_index))

Total 400000 word vectors.


In [22]:
embedding_weights = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, index in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_weights[index] = embedding_vector

In [23]:
embedding_weights.shape

(90279, 100)

In [24]:
embedding_layer=Embedding(input_dim=len(word_index)+1,
                          output_dim=EMBEDDING_DIM,
                          weights=[embedding_weights],
                          input_length=MAX_SEQUENCE_LENGTH,
                          trainable=True)

In [29]:
left_question_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(left_question_input)
l_cov1= Conv1D(100, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_drop1=Dropout(0.5)(l_pool1)
l_cov2 = Conv1D(100, 5, activation='relu')(l_drop1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_drop2=Dropout(0.5)(l_pool2)
'''l_cov3 = Conv1D(128, 5, activation='relu')(l_drop2)
l_pool3 = MaxPooling1D(5)(l_cov3)  # global max pooling
l_drop3=Dropout(0.5)(l_pool3)'''
l_flat = Flatten()(l_drop2)
left_lstm_sent = Dense(100, activation='relu')(l_flat)

right_question_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(right_question_input)
l_cov1= Conv1D(100, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_drop1=Dropout(0.5)(l_pool1)
l_cov2 = Conv1D(100, 5, activation='relu')(l_drop1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_drop2=Dropout(0.5)(l_pool2)
'''l_cov3 = Conv1D(128, 5, activation='relu')(l_drop2)
l_pool3 = MaxPooling1D(5)(l_cov3)  # global max pooling
l_drop3=Dropout(0.5)(l_pool3)'''
l_flat = Flatten()(l_drop2)
right_lstm_sent = Dense(100, activation='relu')(l_flat)

#merge two encoded inputs with the l1 distance between them
L1_distance = lambda x: K.abs(x[0]-x[1])
both = merge([left_lstm_sent,right_lstm_sent], mode = L1_distance, output_shape=lambda x: x[0])
prediction = Dense(2,activation='sigmoid')(both)

siamese_net = Model(input=[left_question_input,right_question_input],output=prediction)
siamese_net.compile(loss="binary_crossentropy",optimizer=Adam())

siamese_net.count_params()

print("model - Siamese Networks CNN dropout:0.5")
print(siamese_net.summary())

model - Siamese Networks CNN dropout:0.5
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 200)           0                                            
____________________________________________________________________________________________________
input_6 (InputLayer)             (None, 200)           0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 200, 100)      9027900     input_5[0][0]                    
                                                                   input_6[0][0]                    
____________________________________________________________________________________________________
convolution1d_9 (Convolution1D)  (None, 196, 100) 

In [30]:
#data_question2_1000_train.shape
#duplicates_1000_train=np.array(duplicates_1000_train).reshape(1000,1)
#duplicates_1000_vali=np.array(duplicates_1000_vali).reshape(1000,1)

In [31]:
# checkpoint
filepath="cnn-2layers-dropout-5-glove-weights-improvement-{epoch:02d}-{val_loss:.3f}.hdf5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint1]
# Fit the model
siamese_net.fit([questions1_sequences_train,questions2_sequences_train], duplicates_train,
                validation_data=([questions1_sequences_val,questions2_sequences_val],duplicates_val)
                                  ,nb_epoch=10, batch_size=50,callbacks=callbacks_list,verbose=0)

Epoch 00000: val_loss improved from inf to 0.49735, saving model to cnn-2layers-dropout-5-glove-weights-improvement-00-0.497.hdf5
Epoch 00001: val_loss improved from 0.49735 to 0.48618, saving model to cnn-2layers-dropout-5-glove-weights-improvement-01-0.486.hdf5
Epoch 00002: val_loss improved from 0.48618 to 0.48210, saving model to cnn-2layers-dropout-5-glove-weights-improvement-02-0.482.hdf5
Epoch 00003: val_loss improved from 0.48210 to 0.47820, saving model to cnn-2layers-dropout-5-glove-weights-improvement-03-0.478.hdf5
Epoch 00004: val_loss did not improve
Epoch 00005: val_loss did not improve
Epoch 00006: val_loss did not improve


KeyboardInterrupt: 